In [1]:
%load_ext autoreload
%autoreload 2

# resample (using original nouns) since we had a bug where we were including full VocabItem rather than just noun


In [19]:
from proj.cxs_are_revealed.paper.data_config import Exp4NPN
from openai import ChatCompletion
from proj.cxs_are_revealed.paper.proj_common.npn_dataset_generation.npn_utils import GPTOutput_with_nounrep, GPTResult
from rozlib.libs.common.data.utils_jsonl import read_from_jsonl
from typing import List
from rozlib.libs.common.data.utils_jsonl import dump_to_jsonl
from proj.cxs_are_revealed.paper.proj_common.npn_dataset_generation.gpt_sample import rungpt_with_prompt
from proj.cxs_are_revealed.paper.proj_common.npns import get_npn_data



In [3]:
# we are going to re-use the same nouns, so read back in
gpt_orig = read_from_jsonl(Exp4NPN.npn_gpt_outputs, GPTOutput_with_nounrep)
gpt_orig[0]



GPTOutput_with_nounrep(noun=NounRep(id=24348, token='Ġbinge', str_rep=' binge', str_rep_no_space='binge'), prep='to', model='gpt-4-0613', output='She was stuck in a dangerous cycle, binge after binge, with no end in sight.', finish_reason='stop')

In [12]:
# count number of original fails
ct = 0
fails: List[GPTOutput_with_nounrep] = []
for go in gpt_orig:
    npn = go.npn.lower()
    if go.output.lower().find(npn) == -1:
        ct+=1
        # print(f"{npn} not in \n\t{go.output})")
        fails.append(go)
print(ct)
print(fails[0])


96


In [21]:
# regenerate
def run_all():
    for idx, go in enumerate(gpt_orig):
        print(idx)
        noun = go.noun.str_rep_no_space
        prep = go.prep
        resp: ChatCompletion = rungpt_with_prompt(noun, prep)
        if not len(resp.choices) == 1:
            print(f"more than one resp")
        gpt_result = GPTResult(
            noun,
            prep,
            resp.model,
            resp.choices[0].message.content,
            resp.choices[0].finish_reason
        )
        dump_to_jsonl(gpt_result, Exp4NPN.npn_gpt_outputs_v2_fixed)

# uncomment to run
# run_all()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [25]:
# read back in and check

npn_data = get_npn_data(
    Exp4NPN.npn_gpt_outputs_v2_fixed,
    None,
    # Exp4NPN.npn_acceptability_ratings_csv,
    output_has_noun_rep=False,
    do_filter_gpt_generations=True
)
print(len(npn_data))


392
notify (non error): more than 2 occurrences! asteroid The explorer traveled from asteroid to asteroid, seeking valuable minerals in the vast expanse of the asteroid belt. 

notify (non error): more than 2 occurrences! asteroid The spacecraft maneuvered through the dense asteroid belt, dodging asteroid upon asteroid in its daring journey. 

notify (non error): more than 2 occurrences! interior As an interior designer, she meticulously worked through interior after interior, bringing each one to life with her unique style. 

notify (non error): more than 2 occurrences! tow As the snowstorm raged on, the tow trucks worked relentlessly, pulling out tow after tow from the heaps of snow. 

notify (non error): more than 2 occurrences! tow The seasoned tow truck driver cleared the pileup on the highway, moving the vehicles tow by tow until the road was clear. 

notify (non error): more than 2 occurrences! comp As he delved deeper into his programming career, he found himself working on com